#########################
# Make sure you are using the `conda_python3` Jupyter Kernel for SageMaker!
## We will install the necessary libraries.

#########################

# TensorFlow Training and Serving in SageMaker "Script Mode"

Script mode is a training script format for TensorFlow that lets you execute any TensorFlow training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native TensorFlow support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). In this example, we will show how easily you can train a SageMaker using TensorFlow 1.x and TensorFlow 2.x scripts with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker TensorFlow Serving container](https://github.com/aws/sagemaker-tensorflow-serving-container). The TensorFlow Serving container is the default inference method for script mode. For full documentation on the TensorFlow Serving container, please visit [here](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst).


# Set up the environment

Let's start by setting up the environment:

In [1]:
import sys
!{sys.executable} -m pip install --upgrade stepfunctions

    100% |████████████████████████████████| 61kB 24.2MB/s ta 0:00:01
Requirement not upgraded as not directly required: sagemaker>=1.42.8 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from stepfunctions) (1.49.0)
Requirement not upgraded as not directly required: boto3>=1.9.213 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from stepfunctions) (1.10.45)
Requirement not upgraded as not directly required: pyyaml in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from stepfunctions) (3.12)
Requirement not upgraded as not directly required: numpy>=1.9.0 in /home/ec2-user/.local/lib/python3.6/site-packages (from sagemaker>=1.42.8->stepfunctions) (1.18.0)
Requirement not upgraded as not directly required: protobuf>=3.1 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from sagemaker>=1.42.8->stepfunctions) (3.11.2)
Requirement not upgraded as not directly required: scipy>=0.19.0 in /home/ec2-user/anaconda3/

## Install SageMaker Python SDK with TensorFlow 2.x Support (>1.49)
_Note:  Ignore Warnings and Errors Below_

In [2]:
!pip install sagemaker --upgrade --ignore-installed --no-cache

    100% |████████████████████████████████| 296kB 48.4MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 62.2MB/s ta 0:00:01
    100% |████████████████████████████████| 20.1MB 71.6MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 71.8MB/s ta 0:00:01
    100% |████████████████████████████████| 26.1MB 57.7MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 50.3MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 68.4MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 54.2MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 69.0MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 72.8MB/s ta 0:00:01
    100% |████████████████████████████████| 122kB 66.1MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 56.5MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 49.6MB/s ta 0:00:01
    100% |████████████████████████████████| 552kB 65.3MB/s ta 0:00:01
    100% |███████████

In [3]:
!pip install tensorflow==2.0.0b1 --upgrade --ignore-installed --no-cache --user

    100% |████████████████████████████████| 87.9MB 69.1MB/s ta 0:00:01  0% |▎                               | 870kB 49.0MB/s eta 0:00:02    52% |████████████████▉               | 46.3MB 47.7MB/s eta 0:00:01
    100% |████████████████████████████████| 1.3MB 76.1MB/s ta 0:00:01
    100% |████████████████████████████████| 501kB 75.5MB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 69.5MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 55.4MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 62.9MB/s ta 0:00:01
    100% |████████████████████████████████| 2.5MB 70.6MB/s ta 0:00:01
    100% |████████████████████████████████| 3.1MB 67.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 57.0MB/s ta 0:00:01
    100% |████████████████████████████████| 20.1MB 46.7MB/s ta 0:00:01    80% |█████████████████████████▊      | 16.2MB 55.6MB/s eta 0:00:01
    100% |████████████████████████████████| 583kB 71.7MB/s ta 0:00:01
    100% |███████████████

In [4]:
!pip list

Package                            Version             
---------------------------------- --------------------
absl-py                            0.9.0               
alabaster                          0.7.10              
anaconda-client                    1.6.14              
anaconda-project                   0.8.2               
asn1crypto                         0.24.0              
astor                              0.8.1               
astroid                            1.6.3               
astropy                            3.0.2               
attrs                              18.1.0              
Automat                            0.3.0               
autovizwidget                      0.13.1              
awscli                             1.16.283            
Babel                              2.5.3               
backcall                           0.1.0               
backports.shutil-get-terminal-size 1.0.0               
bcrypt                             3.1.7        

## Restart the Kernel to Recognize New Dependencies Above

In [5]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)

## Add a policy to your SageMaker role in IAM

**If you are running this notebook on an Amazon SageMaker notebook instance**, the IAM role assumed by your notebook instance needs permission to create and run workflows in AWS Step Functions. To provide this permission to the role, do the following.

1. Open the Amazon [SageMaker console](https://console.aws.amazon.com/sagemaker/). 
2. Select **Notebook instances** and choose the name of your notebook instance
3. Under **Permissions and encryption** select the role ARN to view the role on the IAM console
4. Choose **Attach policies** and search for `AWSStepFunctionsFullAccess`.
5. Select the check box next to `AWSStepFunctionsFullAccess` and choose **Attach policy**

If you are running this notebook in a local environment, the SDK will use your configured AWS CLI configuration. For more information, see [Configuring the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html).

Next, create an execution role in IAM for Step Functions. 

### Create an execution role for Step Functions

You need an execution role so that you can create and execute workflows in Step Functions.

1. Go to the [IAM console](https://console.aws.amazon.com/iam/)
2. Select **Roles** and then **Create role**.
3. Under **Choose the service that will use this role** select **Step Functions**
4. Choose **Next** until you can enter a **Role name**
5. Enter a name such as `StepFunctionsWorkflowExecutionRole` and then select **Create role**


Attach a policy to the role you created. The following steps attach a policy that provides full access to Step Functions, however as a good practice you should only provide access to the resources you need.  

1. Under the **Permissions** tab, click **Add inline policy**
2. Enter the following in the **JSON** tab

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:CreateTransformJob",
                "sagemaker:DescribeTransformJob",
                "sagemaker:StopTransformJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:DescribeTrainingJob",
                "sagemaker:StopTrainingJob",
                "sagemaker:CreateHyperParameterTuningJob",
                "sagemaker:DescribeHyperParameterTuningJob",
                "sagemaker:StopHyperParameterTuningJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateEndpoint",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DeleteEndpoint",
                "sagemaker:UpdateEndpoint",
                "sagemaker:ListTags",
                "lambda:InvokeFunction",
                "sqs:SendMessage",
                "sns:Publish",
                "ecs:RunTask",
                "ecs:StopTask",
                "ecs:DescribeTasks",
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem",
                "batch:SubmitJob",
                "batch:DescribeJobs",
                "batch:TerminateJob",
                "glue:StartJobRun",
                "glue:GetJobRun",
                "glue:GetJobRuns",
                "glue:BatchStopJobRun"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:PutRule",
                "events:DescribeRule"
            ],
            "Resource": [
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTrainingJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTransformJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTuningJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForECSTaskRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForBatchJobsRule"
            ]
        }
    ]
}
```

3. Choose **Review policy** and give the policy a name such as `StepFunctionsWorkflowExecutionPolicy`
4. Choose **Create policy**. You will be redirected to the details page for the role.
5. Copy the **Role ARN** at the top of the **Summary**

### Import the required modules & create the AWS SageMaker execution role

Now import the required modules from the Step Functions SDK and AWS SageMaker, configure an S3 bucket, and get the AWS SageMaker execution role.

In [7]:
import os
import sagemaker
import stepfunctions
import logging

from stepfunctions.template.pipeline import TrainingPipeline

sagemaker_session = sagemaker.Session()
stepfunctions.set_stream_logger(level=logging.INFO)

#bucket = sagemaker_session.default_bucket()
#prefix = 'sagemaker/DEMO-tensorflow-mnist'

# SageMaker Execution Role
# You can use sagemaker.get_execution_role() if running inside sagemaker's notebook instance
sagemaker_execution_role = sagemaker.get_execution_role() #Replace with ARN if not in an AWS SageMaker notebook

# paste the StepFunctionsWorkflowExecutionRole ARN from above
workflow_execution_role = "arn:aws:iam::806570384721:role/StepFunctionsWorkflowExecutionRole" 

## Setup the Service Execution Role and Region
Get IAM role arn used to give training and hosting access to your data.  See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [8]:
#role = get_execution_role()
print('RoleARN:  {}\n'.format(sagemaker_execution_role))

region = sagemaker_session.boto_session.region_name
print('Region:  {}'.format(region))

RoleARN:  arn:aws:iam::806570384721:role/service-role/AmazonSageMaker-ExecutionRole-20191201T115647

Region:  us-east-1


## Training Data

The MNIST dataset has been loaded to the public S3 buckets ``sagemaker-sample-data-<REGION>`` under the prefix ``tensorflow/mnist``.

In [9]:
original_training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)
print(original_training_data_uri)

s3://sagemaker-sample-data-us-east-1/tensorflow/mnist


### Copy the Training Data to Your Notebook Disk

In [10]:
local_data_path = './data'

In [11]:
!aws --region {region} s3 cp --recursive {original_training_data_uri} {local_data_path}

download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/eval_labels.npy to data/eval_labels.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_labels.npy to data/train_labels.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/eval_data.npy to data/eval_data.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_data.npy to data/train_data.npy


There are four ``.npy`` file under this prefix:
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [12]:
!ls {local_data_path}

eval_data.npy  eval_labels.npy	train_data.npy	train_labels.npy


### Upload the Data to S3 for Distributed Training Across Many Workers
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

This is S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.

In [13]:
bucket = sagemaker_session.default_bucket()
data_prefix = 'sagemaker/tensorflow-mnist/data'

In [14]:
training_data_uri = sagemaker_session.upload_data(path=local_data_path, bucket=bucket, key_prefix=data_prefix)
print(training_data_uri)

s3://sagemaker-us-east-1-806570384721/sagemaker/tensorflow-mnist/data


In [15]:
!aws s3 ls --recursive {training_data_uri}

2019-12-27 16:14:35   31360128 sagemaker/tensorflow-mnist/data/eval_data.npy
2019-12-27 16:14:33      40128 sagemaker/tensorflow-mnist/data/eval_labels.npy
2019-12-27 16:14:33  172480128 sagemaker/tensorflow-mnist/data/train_data.npy
2019-12-27 16:14:36     220128 sagemaker/tensorflow-mnist/data/train_labels.npy


## Train
https://sagemaker.readthedocs.io/en/stable/using_tf.html#distributed-training

This tutorial's training script was adapted from TensorFlow's official [CNN MNIST example](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/layers/cnn_mnist.py). We have modified it to handle the ``model_dir`` parameter passed in by SageMaker. This is an S3 path which can be used for data sharing during distributed training and checkpointing and/or model persistence. We have also added an argument-parsing function to handle processing training-related variables.

At the end of the training job we have added a step to export the trained model to the path stored in the environment variable ``SM_MODEL_DIR``, which always points to ``/opt/ml/model``. This is critical because SageMaker uploads all the model artifacts in this folder to S3 at end of training.

### Training Script

In [16]:
!ls ./src/mnist_keras_tf2.py

./src/mnist_keras_tf2.py


You can add custom Python modules to the `src/requirements.txt` file.  They will automatically be installed - and made available to your training script.

In [17]:
!cat ./src/requirements.txt

# Python dependencies go here...

## Use Step Functions to run training in SageMaker


### Train with SageMaker `TensorFlow` Estimator

https://sagemaker.readthedocs.io/en/stable/using_tf.html#distributed-training

The `sagemaker.tensorflow.TensorFlow` estimator handles locating the script mode container, uploading your script to a S3 location and creating a SageMaker training job. Let's call out a couple important parameters here:

* `py_version` is set to `'py3'` to indicate that we are using script mode since legacy mode supports only Python 2. Though Python 2 will be deprecated soon, you can use script mode with Python 2 by setting `py_version` to `'py2'` and `script_mode` to `True`.

* `distributions` is used to configure the distributed training setup. It's required only if you are doing distributed training either across a cluster of instances or across multiple GPUs. Here we are using parameter servers as the distributed training schema. SageMaker training jobs run on homogeneous clusters. To make parameter server more performant in the SageMaker setup, we run a parameter server on every instance in the cluster, so there is no need to specify the number of parameter servers to launch. Script mode also supports distributed training with [Horovod](https://github.com/horovod/horovod). You can find the full documentation on how to configure `distributions` [here](https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#distributed-training). 

Notes:  
* This example uses two(2) `ml.p3.2xlarge` instances.  You will likely need to request a SageMaker instance limit increase from Support before continuing.

* Alternatively, you can specify `ml.c5.2xlarge` instead.

* To recognize the `requirements.txt`, we must include `src/setup.py` per [this](https://github.com/aws/sagemaker-python-sdk/issues/911) GitHub issue.

In [50]:
from sagemaker.tensorflow import TensorFlow

model_output_path = 's3://{}/sagemaker/tensorflow-mnist/training-runs'.format(bucket)

mnist_estimator = TensorFlow(entry_point='mnist_keras_tf2.py',
                             source_dir='./src',
                             output_path=model_output_path,
                             role=sagemaker_execution_role,
                             train_instance_count=2,
                             train_instance_type='ml.p3.2xlarge',
                             framework_version='2.0.0',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}})

### Build a training pipeline with the Step Functions SDK

A typical task for a data scientist is to train a model and deploy that model to an endpoint. Without the Step Functions SDK, this is a four step process on SageMaker that includes the following.

1. Training the model
2. Creating the model on SageMaker
3. Creating an endpoint configuration
4. Deploying the trained model to the configured endpoint

The Step Functions SDK provides the [TrainingPipeline](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/pipelines.html#stepfunctions.template.pipeline.train.TrainingPipeline) API to simplify this procedure. The following configures `pipeline` with the necessary parameters to define a training pipeline.

In [51]:
print(training_data_uri)
print(model_output_path)

s3://sagemaker-us-east-1-806570384721/sagemaker/tensorflow-mnist/data
s3://sagemaker-us-east-1-806570384721/sagemaker/tensorflow-mnist/training-runs


In [52]:
pipeline = TrainingPipeline(
    estimator=mnist_estimator,
    role=workflow_execution_role,
    inputs=training_data_uri,
    s3_bucket=model_output_path
)

AttributeError: 'Model' object has no attribute '__framework_name__'

### Visualize the pipeline

You can now view the workflow definition, and also visualize it as a graph. This workflow and graph represent your training pipeline.

#### View the workflow definition

In [41]:
print(pipeline.workflow.definition.to_json(pretty=True))

NameError: name 'pipeline' is not defined

#### Visualize the workflow graph

In [ ]:
pipeline.render_graph()

### Create and execute the pipeline on AWS Step Functions

Create the pipeline in AWS Step Functions with [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create).

In [ ]:
pipeline.create()

Run the workflow with [execute](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.execute). A link will be provided after the following cell is executed. Following this link, you can monitor your pipeline execution on Step Functions' console.

In [ ]:
pipeline.execute()

# Manually train the model

## `fit` the Model (Approx. 15 mins)

To start a training job, we call `estimator.fit(training_data_uri)`.

An S3 location is used here as the input. `fit` creates a default channel named `'training'`, which points to this S3 location. In the training script we can then access the training data from the location stored in `SM_CHANNEL_TRAINING`. `fit` accepts other parameters, as well. See the API doc [here](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) for details.

When training starts, the TensorFlow container executes `mnist_keras_tf2.py` by passing `hyperparameters` and `model_dir` from the estimator as script arguments. Because we didn't define either in this example, no hyperparameters are passed, and `model_dir` defaults to `s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`, so the script execution is as follows:
```bash
python mnist_keras_tf2.py --model_dir s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>
```
When training is complete, the training job will upload the saved model for TensorFlow serving.

In [ ]:
mnist_estimator.fit(inputs=training_data_uri, wait=False)

training_job_name = mnist_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

After some time, or in a separate Python notebook, we can attach to the running job using the `training_job_name`.

In [ ]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow.attach(training_job_name=training_job_name)

## Option 1:  Perform Batch Predictions Directly in the Notebook

Use TensorFlow Core to load the SavedModel from `model_output_path`

In [ ]:
!aws --region {region} s3 ls --recursive {model_output_path}/{training_job_name}/output/

In [ ]:
!aws --region {region} s3 cp {model_output_path}/{training_job_name}/output/model.tar.gz ./model/model.tar.gz

In [ ]:
!ls ./model

In [ ]:
!tar -xzvf ./model/model.tar.gz -C ./model

Show the signature of the model for prediction purposes

In [ ]:
!saved_model_cli show --dir ./model/000000001 --tag_set serve --signature_def serving_default

In [ ]:
import tensorflow as tf

print(tf.__version__)

loaded = tf.saved_model.load(export_dir='./model/000000001/')
print(list(loaded.signatures.keys()))  # ["serving_default"]

In [ ]:
# TODO:  Finish this

## Option 2:  Perform Batch Predictions with a Batch Transform Job

We can create a [Batch Transformation Job](https://sagemaker.readthedocs.io/en/stable/using_tf.html#run-a-batch-transform-job) to run batch predictions.

In [ ]:
batch_output_uri = '{}/batch_output'.format(training_data_uri) # The location to store the results
print(batch_output_uri)

In [ ]:
mnist_transformer = mnist_estimator.transformer(instance_count=1, instance_type='ml.p3.2xlarge', output_path=batch_output_uri)

TODO:  We may need to use a custom data transformer to convert numpy format into json

In [ ]:
mnist_transformer.transform(data=training_data_uri, data_type='S3Prefix', content_type='text/csv', split_type='Line', wait=False)

transform_job_name = mnist_transformer.latest_transformer_job.name
print('transform_job_name:  {}'.format(transform_job_name))

#print(batch_transform_job_name)
#print(mnist_transformer.job_name)

In [ ]:
mnist_transformer.attach(transform_job_name=transform_job_name)

In [ ]:
!aws --region {region} s3 ls --recursive {batch_output_uri}

## Option 3:  Create a SageMaker Endpoint and Perform REST-based Predictions

### Deploy the Trained Model to a SageMaker Endpoint (Approx. 10 mins)

After training, we use the `TensorFlow` estimator object to build and deploy a `TensorFlowPredictor`.  This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `tensorflow_mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-tensorflow-containers](https://github.com/aws/sagemaker-tensorflow-containers).

The `deploy()` method creates a SageMaker model, which is then deployed to an endpoint to serve prediction requests in real time.  We will use the TensorFlow Serving container for the endpoint, because we trained with script mode.  This serving container runs an implementation of a web server that is compatible with SageMaker hosting protocol.  The [Using your own inference code](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) document explains how SageMaker runs inference containers.

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint.  These do not need to be the same as the values we used for the training job.  For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`.  Here we will deploy the model to a single `ml.p3.2xlarge` instance.  Alternatively, you can use a `ml.c5.2xlarge` instance.

In [ ]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

### Invoke the Endpoint

Let's download the training data and use that as input for inference.

In [ ]:
import numpy as np

train_data = np.load('{}/train_data.npy'.format(local_data_path))
train_labels = np.load('{}/train_labels.npy'.format(local_data_path))

The formats of the input and the output data correspond directly to the request and response formats of the `Predict` method in the [TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest). SageMaker's TensforFlow Serving endpoints can also accept additional input formats that are not part of the TensorFlow REST API, including the simplified JSON format, line-delimited JSON objects ("jsons" or "jsonlines"), and CSV data.

In this example we are using a `numpy` array as input, which will be serialized into the simplified JSON format. In addition, TensorFlow Serving can also process multiple items at once as you can see in the following code. You can find the complete documentation on how to make predictions against a TensorFlow Serving SageMaker endpoint [here](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint).

Examine the prediction result from the TensorFlow 2.0 model.

In [ ]:
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = predictions['predictions'][i]
    label = train_labels[i]
    print('prediction is {}, label is {}, matched: {}'.format(prediction, label, prediction == label))

## Cleanup

Let's delete the endpoint we just created to prevent incurring any extra costs.

In [ ]:
# mnist_estimator.delete_endpoint()

sagemaker.Session().delete_endpoint(predictor.endpoint)